In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
# Make the data set and labels
# Create feature vector num. of samples x num. of features
#X= np.array([[-3,7],[1,5], [1,2], [-2,0], [2,3], [-4,0], [-1,1], [1,1], [-2,2], [2,7], [-4,1], [-2,7]])
# Create feature vector num. of samples x 1 (bot or not)
#y = np.array([3, 3, 3, 3, 4, 3, 3, 4, 3, 4, 4, 4])
import pickle
import pandas as pd
#X_b = pd.read_pickle("bot_features.pkl")
#X_n = pd.read_pickle("nonbot_features.pkl")
X_b = np.load(open('bot_features.npy','rb'))
X_n = np.load(open('nonbot_features.npy','rb'))
X_b = [ row[1:] for row in X_b ]
X_n = [ row[1:] for row in X_n ]

# After the Data is Processed the data is 75/25 split for train/test

In [2]:
# Split the data set for all classifiers first
from sklearn.model_selection import train_test_split
X_train = []
X_test = []
y_train = []
y_test = []
y_b = np.ones(len(X_b))
y_n = np.zeros(len(X_n))
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_b, y_b, test_size=0.25, random_state=42)
X_train.extend(X_train_b)
X_test.extend(X_test_b)
y_train.extend(y_train_b)
y_test.extend(y_test_b)
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_n, y_n, test_size=0.25, random_state=24)
X_train.extend(X_train_n)
X_test.extend(X_test_n)
y_train.extend(y_train_n)
y_test.extend(y_test_n)


# Gaussian Naive Bayes 

In [3]:
from sklearn.naive_bayes import GaussianNB
# Naive Bayes for tweets
# Create a Gaussian Classifier
clf_NB = GaussianNB()
# Train the model using the training sets 
clf_NB.fit(X_train, y_train)
#Predict Output 
predicted_NB = clf_NB.predict(X_test)
y_true = y_test
y_pred = predicted_NB
print('For Gaussian Naive Bayes:')
acc = accuracy_score(y_true, y_pred)
print ("The accuracy is: %f%% " % ( 100*acc ))
auc = (roc_auc_score(y_true, y_pred))
print ("The AUC score is: %f " %  ( auc ))
pre = metrics.precision_score(y_true, y_pred)
print ("The precision is: %f " %  ( pre ))
rec = metrics.recall_score(y_true, y_pred)
print ("The recall is: %f " %  ( rec ))
f1 = metrics.f1_score(y_true, y_pred)  
print ("The f1 score is: %f " %  ( f1 ))

For Gaussian Naive Bayes:
The accuracy is: 79.213483% 
The AUC score is: 0.799382 
The precision is: 0.717391 
The recall is: 0.920319 
The f1 score is: 0.806283 


# Decison Tree

In [4]:
# Decison tree for tweets
# Think about using PCA to simplfy data and have better DT
# To find best Tree depth
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from math import floor
# Should test depth from 3 to num. of features
#param_grid = {'max_depth': np.arange(3, len(X_train[1])), 'min_samples_split':np.floor(len(y_train)*(np.arange(1,10,.5)/100))}
param_grid = {'max_depth': np.arange(3, len(X_train[1])), 'min_samples_split':(np.arange(.5,10,.5)/100)}
#param_grid = {'max_depth': np.arange(3, len(X_train[1])), 'min_samples_split':[5,10,12]}
clf_DT = GridSearchCV(tree.DecisionTreeClassifier(), param_grid)
clf_DT = clf_DT.fit(X_train, y_train)
#predicted_DT = clf_DT.predict_proba(xtest)[:, 1]
predicted_DT = clf_DT.predict(X_test)
best_set = clf_DT.best_params_
y_true = y_test
y_pred = predicted_DT
print('For Decision Tree:')
acc = accuracy_score(y_true, y_pred)
print ("The accuracy is: %f%% " % ( 100*acc ))
auc = (roc_auc_score(y_true, y_pred))
print ("The AUC score is: %f " %  ( auc ))
pre = metrics.precision_score(y_true, y_pred)
print ("The precision is: %f " %  ( pre ))
rec = metrics.recall_score(y_true, y_pred)
print ("The recall is: %f " %  ( rec ))
f1 = metrics.f1_score(y_true, y_pred)  
print ("The f1 score is: %f " %  ( f1 ))
print(best_set)

For Decision Tree:
The accuracy is: 89.325843% 
The AUC score is: 0.891860 
The precision is: 0.900826 
The recall is: 0.868526 
The f1 score is: 0.884381 
{'max_depth': 8, 'min_samples_split': 0.0050000000000000001}


In [46]:
clf_DT2 = tree.DecisionTreeClassifier(max_depth=8,min_samples_split= 0.014999999999999999)
clf_DT2 = clf_DT2.fit(X_train, y_train)
# To visualize the tree
from IPython.display import Image  
import pydotplus
names = ['has_bot_string','account_age','followers','favourites','contributors','friends','geo','has_extended_profile','is_translation_enabled','lang','location','notifications','tweets','verified','url','default_profile','default_profile_image','listed']
clss = ['bot','not_bot']
dot_data = tree.export_graphviz(clf_DT2, out_file=None, 
                     feature_names=names,  
                     class_names=clss,  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())  

InvocationException: GraphViz's executables not found

# Random Forest

In [27]:
# Random Forests
from sklearn.ensemble import RandomForestClassifier
clf_RF = RandomForestClassifier()
# param_grid = {"max_depth": [np.arange(3, len(X_train[1])), None],
#               "min_samples_split": (len(y_train)*(np.arange(0,11,.5)/100)),
#               "bootstrap": [True, False],
#               "criterion": ["gini", "entropy"]}
param_grid = {"max_depth": [np.arange(3, len(X_train[1])), None],
              "min_samples_split": (np.arange(.5,10,.5)/100),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
grid_search_RF = GridSearchCV(clf_RF, param_grid=param_grid)
grid_search_RF.fit(X_train, y_train)
predicted_RF = grid_search_RF.predict(X_test)
y_pred = predicted_RF
print('For Random Forest:')
acc = accuracy_score(y_true, y_pred)
print ("The accuracy is: %f%% " % ( 100*acc ))
auc = (roc_auc_score(y_true, y_pred))
print ("The AUC score is: %f " %  ( auc ))
pre = metrics.precision_score(y_true, y_pred)
print ("The precision is: %f " %  ( pre ))
rec = metrics.recall_score(y_true, y_pred)
print ("The recall is: %f " %  ( rec ))
f1 = metrics.f1_score(y_true, y_pred)  
print ("The f1 score is: %f " %  ( f1 ))
best_set_RF = grid_search_RF.best_params_
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
report(grid_search.cv_results_)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

# Single SVM

In [7]:
# SVM
# recommended to scale your data. For example, scale each attribute on the input vector X to [0,1] or [-1,+1], 
# or standardize it to have mean 0 and variance 1
from sklearn.svm import SVC
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                    {'kernel': ['poly'], 'degree' : np.arange(1,6), 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]},
                    {'kernel': ['sigmoid'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}]
scores = ['precision', 'recall']
score = scores[1]
clf_SVM = GridSearchCV(SVC(C=1), tuned_parameters, cv=5, scoring='%s_macro' % score)
clf_SVM.fit(X_train, y_train)
best_set_SVM = clf_SVM.best_params_
predicted_SVM = clf_SVM.predict(X_test)
y_pred = predicted_SVM
print('For SVM:')
acc = accuracy_score(y_true, y_pred)
print ("The accuracy is: %f%% " % ( 100*acc ))
auc = (roc_auc_score(y_true, y_pred))
print ("The AUC score is: %f " %  ( auc ))
pre = metrics.precision_score(y_true, y_pred)
print ("The precision is: %f " %  ( pre ))
rec = metrics.recall_score(y_true, y_pred)
print ("The recall is: %f " %  ( rec ))
f1 = metrics.f1_score(y_true, y_pred)  
print ("The f1 score is: %f " %  ( f1 ))

For SVM:
The accuracy is: 88.202247% 
The AUC score is: 0.884638 
The precision is: 0.838129 
The recall is: 0.928287 
The f1 score is: 0.880907 


# Ensemble SVM

In [14]:
# EnsembleSVM
# Want to randomly sample data np.random.shuffle(x)
# And then trian multiple SVM's 
# Hard vote on a label
#print(best_set_SVM)
from math import floor
from sklearn.svm import SVC
#best_set_SVM = C=10, kernel=linear,
n_sets = 10
#nsn = floor(len(X_train_n)/n_sets)
#nsb = floor(len(X_train_b)/n_sets)
svm1 = SVC(C=10, kernel='linear', random_state=1)
svm2 = SVC(C=10, kernel='linear', random_state=2)
svm3 = SVC(C=10, kernel='linear', random_state=3)
svm4 = SVC(C=10, kernel='linear', random_state=4)
svm5 = SVC(C=10, kernel='linear', random_state=5)
svm6 = SVC(C=10, kernel='linear', random_state=6)
svm7 = SVC(C=10, kernel='linear', random_state=7)
svm8 = SVC(C=10, kernel='linear', random_state=8)
svm9 = SVC(C=10, kernel='linear', random_state=9)
svm10 = SVC(C=10, kernel='linear', random_state=10)
sb = np.array_split(X_train_b,n_sets)
sn = np.array_split(X_train_n,n_sets)
sbc = np.array_split(y_train_b,n_sets)
snc = np.array_split(y_train_n,n_sets)
for i in range(0,n_sets):
    sb[i]=np.concatenate((sb[i], sn[i]), axis=0)
    sbc[i]=np.concatenate((sbc[i], snc[i]), axis=0)
svm1.fit(sb[0],sbc[1])
svm2.fit(sb[1],sbc[2])
svm3.fit(sb[2],sbc[3])
svm4.fit(sb[3],sbc[4])
svm5.fit(sb[4],sbc[5])
svm6.fit(sb[5],sbc[6])
svm7.fit(sb[6],sbc[6])
svm8.fit(sb[7],sbc[7])
svm9.fit(sb[8],sbc[8])
svm10.fit(sb[9],sbc[9])
vt1 = svm1.predict(X_test)
vt2 = svm2.predict(X_test)
vt3 = svm3.predict(X_test)
vt4 = svm4.predict(X_test)
vt5 = svm5.predict(X_test)
vt6 = svm6.predict(X_test)
vt7 = svm7.predict(X_test)
vt8 = svm8.predict(X_test)
vt9 = svm9.predict(X_test)
vt10 = svm10.predict(X_test)
vt_count = np.array(vt1)+np.array(vt2)+np.array(vt3)+np.array(vt4)+np.array(vt5)+np.array(vt6)+np.array(vt7)+np.array(vt8)+np.array(vt9)+np.array(vt10)
predicted_ES = [ vt_count[i]>=5 for i in range(0,len(vt_count)) ]
y_pred = predicted_ES
print('For Ensemble SVM:')
acc = accuracy_score(y_true, y_pred)
print ("The accuracy is: %f%% " % ( 100*acc ))
auc = (roc_auc_score(y_true, y_pred))
print ("The AUC score is: %f " %  ( auc ))
pre = metrics.precision_score(y_true, y_pred)
print ("The precision is: %f " %  ( pre ))
rec = metrics.recall_score(y_true, y_pred)
print ("The recall is: %f " %  ( rec ))
f1 = metrics.f1_score(y_true, y_pred)  
print ("The f1 score is: %f " %  ( f1 ))

ValueError: Found input variables with inconsistent numbers of samples: [161, 160]

# Hybrid Ensemble: SVM and Random Forest

In [16]:
# Hybrid Ensemble
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
clf1 = RandomForestClassifier( random_state=1)
clf2 = SVC(C=10, kernel='linear', random_state=1)
eclf1 = VotingClassifier(estimators=[('RF', clf1), ('SVM', clf2)], voting='hard')
# hard = majority vote, soft = argmax of the sums of the predicted probabilities
eclf1 = eclf1.fit(X_train,y_train)
predicted_ens = eclf1.predict(X_test)
y_pred = predicted_ens
print('For Hybrid Ensemble:')
acc = accuracy_score(y_true, y_pred)
print ("The accuracy is: %f%% " % ( 100*acc ))
auc = (roc_auc_score(y_true, y_pred))
print ("The AUC score is: %f " %  ( auc ))
pre = metrics.precision_score(y_true, y_pred)
print ("The precision is: %f " %  ( pre ))
rec = metrics.recall_score(y_true, y_pred)
print ("The recall is: %f " %  ( rec ))
f1 = metrics.f1_score(y_true, y_pred)  
print ("The f1 score is: %f " %  ( f1 ))

For Hybrid Ensemble:
The accuracy is: 89.138577% 
The AUC score is: 0.889192 
The precision is: 0.910638 
The recall is: 0.852590 
The f1 score is: 0.880658 
